<a href="https://colab.research.google.com/github/royajafari/PeopleReplacement/blob/main/Seating_of_guests_inGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim # برای وارد کردن ماژول های بهینه سازی
import numpy as np

# 1. ایجاد داده‌ها
num_people = 400
group_size = 5
num_groups = num_people // group_size
names = [f"Person_{i}" for i in range(num_people)]

# شافل کردن افراد و ایجاد گروه‌ها
np.random.shuffle(names)
groups = [names[i * group_size:(i + 1) * group_size] for i in range(num_groups)]

# ایجاد ماتریس اختلاف
problem_matrix = np.zeros((num_people, num_people))
indices = np.triu_indices(num_people, 1)
selected_indices = np.random.choice(np.arange(len(indices[0])), 600, replace=False)
for idx in selected_indices:
    i, j = indices[0][idx], indices[1][idx]
    problem_matrix[i, j] = problem_matrix[j, i] = 1  # وزن اختلاف‌ها برابر با 1

# اطمینان حاصل کردن از صفر بودن قطر
np.fill_diagonal(problem_matrix, 0)

# تبدیل ماتریس‌ها به PyTorch tensors
problem_matrix = torch.tensor(problem_matrix, dtype=torch.float32)

# نمایش ماتریس اسامی و گروه‌ها
print("Matrix of Names:")
print(names)
print("\nGroups:")
for idx, group in enumerate(groups):
    print(f"Group {idx + 1}: {group}")

# 2. تعریف مدل شبکه عصبی
class SeatingModel(nn.Module):
    def __init__(self, num_people):
        super(SeatingModel, self).__init__()
        self.fc1 = nn.Linear(num_people, 256)  # لایه خطی اول با 256 نورون
        self.fc2 = nn.Linear(256, 128)        # لایه خطی دوم با 128 نورون
        self.fc3 = nn.Linear(128, num_people)  # لایه خروجی با 400 نورون
        self.relu = nn.ReLU()  # تابع فعال‌سازی ReLU

    def forward(self, x):
        x = self.relu(self.fc1(x))  # عبور از لایه اول و اعمال ReLU
        x = self.relu(self.fc2(x))  # عبور از لایه دوم و اعمال ReLU
        x = self.fc3(x)             # عبور از لایه خروجی
        return x

model = SeatingModel(num_people)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# تعریف تابع softmax و انتخاب صندلی‌ها بر اساس احتمالات توزیع شده
def choose_seating(output):
    softmax = nn.Softmax(dim=1)
    prob = softmax(output)
    _, seating_indices = torch.max(prob, dim=1)
    return seating_indices

# تعریف تابع هزینه با استفاده از softmax
def custom_loss(output, problem_matrix, alpha):
    softmax = nn.Softmax(dim=1)
    prob = softmax(output)

    total_loss = 0.0
    for i in range(num_people):
        for j in range(i + 1, num_people):
            if problem_matrix[i, j] > 0:
                prob_diff = torch.abs(prob[i] - prob[j])
                loss_ij = alpha * problem_matrix[i, j] * prob_diff.sum() + (1 - alpha) * problem_matrix[i, j]
                total_loss += loss_ij

    return total_loss.sum()  # اطمینان از اینکه خروجی یک اسکالر است

# آموزش مدل
num_epochs = 15
alpha = 0.7  # تنظیم هایپرپارامتر بین 0 و 1 / حرکت به سمت 0 به معنی تمایل به گروه مانده بیشتر و حرکت به سمت 1 به معنی تمایل به کمتربودن اختلاف
seating = torch.eye(num_people).float()  # صندلی اولیه به صورت ماتریس همانی

for epoch in range(num_epochs):
    optimizer.zero_grad()

    output = model(seating)
    loss = custom_loss(output, problem_matrix, alpha)

    loss.backward()
    optimizer.step()

    # نمایش مقدار تابع هزینه در هر دور
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    # نمایش چیدمان فعلی صندلی‌ها
    if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
        print("Current seating arrangement:")
        current_seating = choose_seating(output).detach().numpy()
        assigned_seats = set()
        seating_arrangement = {}
        for i in range(num_people):
            seat_num = current_seating[i]
            if seat_num in assigned_seats:
                # پیدا کردن یک صندلی جدید برای این فرد
                for new_seat in range(num_people):
                    if new_seat not in assigned_seats:
                        seat_num = new_seat
                        break
            assigned_seats.add(seat_num)
            seating_arrangement[seat_num] = (names[i], [idx for idx, group in enumerate(groups) if names[i] in group][0] + 1)

        for seat in sorted(seating_arrangement.keys()):
            name, group = seating_arrangement[seat]
            print(f"Seat {seat + 1}: {name} (Group {group})")
        print("\n")

# نمایش چیدمان نهایی
output = model(seating)
final_seating = choose_seating(output).detach().numpy()
assigned_seats = set()
final_seating_arrangement = {}
for i in range(num_people):
    seat_num = final_seating[i]
    if seat_num in assigned_seats:
        for new_seat in range(num_people):
            if new_seat not in assigned_seats:
                seat_num = new_seat
                break
    assigned_seats.add(seat_num)
    final_seating_arrangement[seat_num] = (names[i], [idx for idx, group in enumerate(groups) if names[i] in group][0] + 1)

print("Final seating arrangement:")
for seat in sorted(final_seating_arrangement.keys()):
    name, group = final_seating_arrangement[seat]
    print(f"Seat {seat + 1}: {name} (Group {group})")

# محاسبه و نمایش مقدار تابع هزینه در نهایت
final_loss = custom_loss(output, problem_matrix, alpha)
print(f"\nFinal loss: {final_loss.item():.4f}")


Matrix of Names:
['Person_259', 'Person_143', 'Person_360', 'Person_315', 'Person_193', 'Person_292', 'Person_116', 'Person_295', 'Person_51', 'Person_6', 'Person_399', 'Person_199', 'Person_252', 'Person_23', 'Person_205', 'Person_387', 'Person_220', 'Person_29', 'Person_348', 'Person_312', 'Person_333', 'Person_113', 'Person_19', 'Person_246', 'Person_384', 'Person_44', 'Person_103', 'Person_397', 'Person_293', 'Person_238', 'Person_185', 'Person_41', 'Person_34', 'Person_358', 'Person_367', 'Person_239', 'Person_134', 'Person_202', 'Person_274', 'Person_228', 'Person_175', 'Person_386', 'Person_337', 'Person_178', 'Person_63', 'Person_160', 'Person_222', 'Person_209', 'Person_338', 'Person_84', 'Person_261', 'Person_194', 'Person_307', 'Person_262', 'Person_74', 'Person_173', 'Person_124', 'Person_237', 'Person_15', 'Person_52', 'Person_266', 'Person_355', 'Person_356', 'Person_7', 'Person_226', 'Person_325', 'Person_90', 'Person_303', 'Person_186', 'Person_208', 'Person_353', 'Pers